In [ ]:
import glob
import os
from datetime import datetime

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from model import DGCNN
from sklearn import preprocessing
from utils import generating_data
from glob import glob

lr_dgcnn = 0.01
weight_decay_dgcnn = 5e-4 #tem no codigo do GMSS tmbm
K=2
epochs=20 # DEFFERARD TMBM USA 20
device = 'cpu'

hiperparam_run = {
    'lr_dgcnn ':lr_dgcnn,
    'weight_decay_dgcnn':weight_decay_dgcnn,
    'K':K,
    'epochs':epochs
}

label_list = np.array([1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1]) + 1 # -1 for 
#fn = 'de_LDS'
feature_list = ['de_LDS','psd_LDS', 'rasm_LDS', 'dasm_LDS', 'asm_LDS','dcau_LDS']

full_path = "/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures"
all_files = [each_file for each_file in glob(full_path+'/*.mat') if 'label.mat' not in each_file]
all_files.sort()
# all_files = glob.glob(full_path+"*")

all_acc_results = []

In [ ]:
# for cur_file in all_files:
cur_file = all_files[0]
# for fn in feature_list:
fn = 'dasm_LDS'
#item = "15_20130709.mat"
#item = "13_20140527.mat"
item = os.path.basename(cur_file)
cur_user = item.split("_")[0]
user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")

# cur_file = os.path.join(full_path, item)
all_data = sio.loadmat(cur_file)


train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)


In [ ]:
train_data.shape[1]

# Treino completo

In [1]:
import glob
import os
from datetime import datetime

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from model import DGCNN
from sklearn import preprocessing
from utils import generating_data
from glob import glob

lr_dgcnn = 0.01
weight_decay_dgcnn = 5e-4 #tem no codigo do GMSS tmbm
K=2
epochs=20 # DEFFERARD TMBM USA 20
device = 'cpu'

hiperparam_run = {
    'lr_dgcnn ':lr_dgcnn,
    'weight_decay_dgcnn':weight_decay_dgcnn,
    'K':K,
    'epochs':epochs
}

label_list = np.array([1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1]) + 1 # -1 for 
#fn = 'de_LDS'
feature_list = ['de_LDS','psd_LDS', 'rasm_LDS', 'dasm_LDS', 'asm_LDS','dcau_LDS']

full_path = "/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures"
all_files = [each_file for each_file in glob(full_path+'/*.mat') if 'label.mat' not in each_file]
all_files.sort()
# all_files = glob.glob(full_path+"*")

all_acc_results = []
for cur_file in all_files:
    for fn in feature_list:
        #item = "15_20130709.mat"
        #item = "13_20140527.mat"
        item = os.path.basename(cur_file)
        cur_user = item.split("_")[0]
        user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
        print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")

        # cur_file = os.path.join(full_path, item)
        all_data = sio.loadmat(cur_file)


        train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)

        #train_data = preprocessing.scale(train_data)
        #test_data = preprocessing.scale(test_data)

        train_data_tensor = torch.from_numpy(train_data).to(torch.float)#.to(device)
        test_data_tensor = torch.from_numpy(test_data).to(torch.float)#.to(device)
        train_label_tensor = torch.from_numpy(train_label).to(torch.long)#.to(device)
        test_label_tensor = torch.from_numpy(test_label).to(torch.long)#.to(device)


        # define model
        model = DGCNN(
            in_channels=5, #5 frequencias
            num_electrodes=train_data.shape[1], #depende de cada feature
            k_adj=K, #numero de layers - copiando do GMSS - na verdade no GMSS K=ordem de chebyshev
            # out_channels=32, #copiando do GMSS, mas ainda nao entendi
            out_channels=32,
            num_classes=3
        )

        best_test_res = {
            'acc':0,
            'predict_label':None,
            'trur_label':None
        }

        optimizer = optim.Adam(model.parameters(),
                            lr=lr_dgcnn, weight_decay=weight_decay_dgcnn)
        myloss = nn.CrossEntropyLoss()

        for cur_epoch in range(epochs):
            # ordem 1
            # model.train() # como apontar  referencia aos dados de treino? R: E na definicao do modelo
            # optimizer.zero_grad() #etapa comum em todos, zerar o gradiente.

            # output = model(train_data_tensor)
            # #TODO accuracy.
            # #print(output)

            # # import torch.nn.functional as F
            # # loss_train = F.nll_loss(output, train_label)

            # myloss = nn.CrossEntropyLoss()
            # loss = myloss(output, train_label_tensor)
            # loss.backward()
            # optimizer.step()

            # ordem 2
            model.train() # como apontar  referencia aos dados de treino? R: E na definicao do modelo
            output = model(train_data_tensor)
            # STEP 10 - Calculate the loss function
            loss = myloss(output, train_label_tensor)
            
            optimizer.zero_grad() #etapa comum em todos, zerar o gradiente.

            #TODO accuracy.
            #print(output)

            # import torch.nn.functional as F
            # loss_train = F.nll_loss(output, train_label)
            # STEP 11 - Updating the Adj matrix
            loss.backward()
            optimizer.step()
            # Estou sentindo falta do coidgo de atualizacao dessa matriz

            train_acc = torch.sum(torch.argmax(output, 1) == train_label_tensor) / train_data_tensor.shape[0]

            print('Epoch : {} -- TrainLoss : {} -- TrainAcc : {}\n'.format(cur_epoch, loss.cpu().data, train_acc.cpu().data))

        # model.eval()
        test_output = model(test_data_tensor)
        test_loss = myloss(test_output, test_label_tensor)
        test_acc = torch.sum(torch.argmax(test_output, 1) == test_label_tensor) / test_data_tensor.shape[0]
        print('Epoch : {} -- TestLoss : {} -- TestAcc : {}\n'.format(cur_epoch, test_loss.cpu().data, test_acc.cpu().data))

        # save the best results    
        # if best_test_res['acc'] < test_acc.cpu().data:
        best_test_res['acc'] = test_acc.cpu().data 
        best_test_res['loss'] = test_loss.cpu().data 
        best_test_res['predict_label'] = torch.argmax(test_output, 0).cpu().numpy()
        best_test_res['true_label'] = test_label_tensor.cpu().numpy()
        print('update res')

        all_acc_results.append([fn,item,best_test_res, cur_file])

        print(best_test_res)



/home/joaovmrf/miniconda3/envs/ppget_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- reading file: 10_20131130.mat
--user:10, sessions: 2013-11-30 00:00:00
	### TRAIN ###
		de_LDS1 - video 1 - (235, 62, 5) observacoes
		de_LDS2 - video 2 - (233, 62, 5) observacoes
		de_LDS3 - video 3 - (206, 62, 5) observacoes
		de_LDS4 - video 4 - (238, 62, 5) observacoes
		de_LDS5 - video 5 - (185, 62, 5) observacoes
		de_LDS6 - video 6 - (195, 62, 5) observacoes
		de_LDS7 - video 7 - (237, 62, 5) observacoes
		de_LDS8 - video 8 - (216, 62, 5) observacoes
		de_LDS9 - video 9 - (265, 62, 5) observacoes
	### TEST ###
		de_LDS10 - video 10 - (237, 62, 5) observacoes
		de_LDS11 - video 11 - (235, 62, 5) observacoes
		de_LDS12 - video 12 - (233, 62, 5) observacoes
		de_LDS13 - video 13 - (235, 62, 5) observacoes
		de_LDS14 - video 14 - (238, 62, 5) observacoes
		de_LDS15 - video 15 - (206, 62, 5) observacoes
Epoch : 0 -- TrainLoss : 1.2576981782913208 -- TrainAcc : 0.3154228925704956

Epoch : 1 -- TrainLoss : 1.6689705848693848 -- TrainAcc : 0.5786069631576538

Epoch : 2 -- TrainLoss : 

In [2]:
all_acc_results

[['de_LDS',
  '10_20131130.mat',
  {'acc': tensor(0.7139),
   'predict_label': array([1333,  468,  236]),
   'trur_label': None,
   'loss': tensor(0.9192),
   'true_label': array([2, 2, 2, ..., 0, 0, 0])},
  '/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures/10_20131130.mat'],
 ['psd_LDS',
  '10_20131130.mat',
  {'acc': tensor(0.6344),
   'predict_label': array([1332,  466,  184]),
   'trur_label': None,
   'loss': tensor(3.0376),
   'true_label': array([2, 2, 2, ..., 0, 0, 0])},
  '/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures/10_20131130.mat'],
 ['rasm_LDS',
  '10_20131130.mat',
  {'acc': tensor(0.5802),
   'predict_label': array([832, 648, 184]),
   'trur_label': None,
   'loss': tensor(1.4062),
   'true_label': array([2, 2, 2, ..., 0, 0, 0])},
  '/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures/10_20131130.mat'],
 ['dasm_LDS',
  '10_20131130.mat',
  {'acc': tensor(0.5614),
   'predict_label': array([842

In [3]:
import pandas as pd

In [12]:
gcnn_result_df = pd.DataFrame(all_acc_results)
gcnn_result_df

,0,1,2,3
0,de_LDS,10_20131130.mat,"{'acc': tensor(0.7139), 'predict_label': [1333...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...
1,psd_LDS,10_20131130.mat,"{'acc': tensor(0.6344), 'predict_label': [1332...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...
2,rasm_LDS,10_20131130.mat,"{'acc': tensor(0.5802), 'predict_label': [832,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...
3,dasm_LDS,10_20131130.mat,"{'acc': tensor(0.5614), 'predict_label': [842,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...
4,asm_LDS,10_20131130.mat,"{'acc': tensor(0.5542), 'predict_label': [832,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...
...,...,...,...,...
265,psd_LDS,9_20140704.mat,"{'acc': tensor(0.8374), 'predict_label': [1382...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...
266,rasm_LDS,9_20140704.mat,"{'acc': tensor(0.7825), 'predict_label': [892,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...
267,dasm_LDS,9_20140704.mat,"{'acc': tensor(0.7355), 'predict_label': [885,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...
268,asm_LDS,9_20140704.mat,"{'acc': tensor(0.7522), 'predict_label': [896,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...


In [14]:
#pd.DataFrame(gcnn_result_df[2].to_list())

In [15]:
gcnn_result_df[["user", "session"]]  = pd.DataFrame(gcnn_result_df[1].str.split('_').to_list())
gcnn_result_df[['acc','predict_label','trur_label','loss','true_label']] = pd.DataFrame(gcnn_result_df[2].to_list())
gcnn_result_df

,0,1,2,3,user,session,acc,predict_label,trur_label,loss,true_label
0,de_LDS,10_20131130.mat,"{'acc': tensor(0.7139), 'predict_label': [1333...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,10,20131130.mat,tensor(0.7139),"[1333, 468, 236]",None,tensor(0.9192),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,psd_LDS,10_20131130.mat,"{'acc': tensor(0.6344), 'predict_label': [1332...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,10,20131130.mat,tensor(0.6344),"[1332, 466, 184]",None,tensor(3.0376),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,rasm_LDS,10_20131130.mat,"{'acc': tensor(0.5802), 'predict_label': [832,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,10,20131130.mat,tensor(0.5802),"[832, 648, 184]",None,tensor(1.4062),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,dasm_LDS,10_20131130.mat,"{'acc': tensor(0.5614), 'predict_label': [842,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,10,20131130.mat,tensor(0.5614),"[842, 647, 195]",None,tensor(1.3131),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,asm_LDS,10_20131130.mat,"{'acc': tensor(0.5542), 'predict_label': [832,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,10,20131130.mat,tensor(0.5542),"[832, 640, 195]",None,tensor(1.0301),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...,...,...,...,...
265,psd_LDS,9_20140704.mat,"{'acc': tensor(0.8374), 'predict_label': [1382...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,9,20140704.mat,tensor(0.8374),"[1382, 449, 1177]",None,tensor(0.3769),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
266,rasm_LDS,9_20140704.mat,"{'acc': tensor(0.7825), 'predict_label': [892,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,9,20140704.mat,tensor(0.7825),"[892, 368, 1177]",None,tensor(0.7653),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
267,dasm_LDS,9_20140704.mat,"{'acc': tensor(0.7355), 'predict_label': [885,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,9,20140704.mat,tensor(0.7355),"[885, 365, 1177]",None,tensor(0.8168),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
268,asm_LDS,9_20140704.mat,"{'acc': tensor(0.7522), 'predict_label': [896,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,9,20140704.mat,tensor(0.7522),"[896, 368, 1147]",None,tensor(0.9454),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [16]:
gcnn_result_df['acc'] = gcnn_result_df['acc'].apply(lambda x: x.item())

In [34]:
gcnn_result_df[[0,'user','session','acc']].sort_values([0,'user', 'acc'], ascending=[True,False, False])\
.groupby([0]).agg({'acc':['mean','std']})



acc          
              mean       std
0                           
asm_LDS   0.650048  0.160579
dasm_LDS  0.643626  0.153290
dcau_LDS  0.717389  0.148904
de_LDS    0.755074  0.107755
psd_LDS   0.605010  0.187215
rasm_LDS  0.651670  0.164967

In [18]:
# eda_gcnn = gcnn_result_df[['user', 'session','acc', 'predict_label','trur_label','loss','true_label']].sort_values(['user', 'acc'], ascending=[False, False])
eda_gcnn = gcnn_result_df[['user', 0,'session','acc']].sort_values(['user', 'acc'], ascending=[False, False])

eda_gcnn

,user,0,session,acc
252,9,de_LDS,20140620.mat,0.919075
257,9,dcau_LDS,20140620.mat,0.899566
254,9,rasm_LDS,20140620.mat,0.875723
256,9,asm_LDS,20140620.mat,0.855491
265,9,psd_LDS,20140704.mat,0.837428
...,...,...,...,...
109,1,psd_LDS,20131027.mat,0.320087
118,1,asm_LDS,20131030.mat,0.317919
116,1,rasm_LDS,20131030.mat,0.286850
115,1,psd_LDS,20131030.mat,0.273844


In [ ]:
eda_gcnn.reset_index().to_csv(f"GCNN_WITHOUT_EVAL_20230912.csv")


In [21]:
eda_gcnn.groupby(['user', 0,'session']).max()#.acc.mean()

acc
user 0        session               
1    asm_LDS  20131027.mat  0.880780
              20131030.mat  0.317919
              20131107.mat  0.513728
     dasm_LDS 20131027.mat  0.739884
              20131030.mat  0.251445
...                              ...
9    psd_LDS  20140627.mat  0.651012
              20140704.mat  0.837428
     rasm_LDS 20140620.mat  0.875723
              20140627.mat  0.754335
              20140704.mat  0.782514

[270 rows x 1 columns]

In [ ]:
eda_gcnn.groupby(['user']).head(2).reset_index(drop=True)#.acc.mean()

# analisar dataset

In [ ]:
import glob
import os
from datetime import datetime

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from model import DGCNN
from sklearn import preprocessing
from utils import generating_data
from glob import glob

In [ ]:
fn = 'de_LDS'

label_list = np.array([1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1]) + 1 # -1 for 

full_path = "/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures"
all_files = [each_file for each_file in glob(full_path+'/*.mat') if 'label.mat' not in each_file]
all_files.sort()
# all_files = glob.glob(full_path+"*")


# for cur_file in all_files: 
#item = "15_20130709.mat"
#item = "13_20140527.mat"
cur_file = all_files[10]
item = os.path.basename(cur_file)
cur_user = item.split("_")[0]
user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")

# cur_file = os.path.join(full_path, item)
all_data = sio.loadmat(cur_file)


train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)

In [ ]:
test_data.size

In [ ]:
test_data_tensor = torch.from_numpy(test_data).to(torch.float)#.to(device)
test_data_tensor.size()

In [ ]:
# pegando apenas o primeiro video (236), todos os canais, a ultima frequencia
test_data_tensor[:236,:,4]

In [ ]:
# pegando apenas o primeiro video (236), canal 1, frequencia delta
test_data_tensor[:236,0,0].size()

# Mudando eixo para entneder matriz de features

In [ ]:
test_data.shape

In [ ]:
test_data_swapped = np.swapaxes(test_data, 0, 1)
test_data_swapped = np.swapaxes(test_data_swapped, 1, 2)
test_data_swapped.shape

In [ ]:
test_data_swapped_tensor = torch.from_numpy(test_data_swapped).to(torch.float)#.to(device)


In [ ]:
test_data_swapped_tensor.size()

In [ ]:
# pegando todos os 62 canais, apenas frequencia delta 
test_data_swapped_tensor[:,0,:].size()